In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, exc
import urllib
import logging

# Logging configuration for professional monitoring
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def run_etl_process():
    try:
        # 1. SQL Server Connection
        params = urllib.parse.quote_plus(
            r'DRIVER={ODBC Driver 17 for SQL Server};'
            r'SERVER=localhost\SQLEXPRESS01;'
            r'DATABASE=AdventureWorksDW2025;'
            r'Trusted_Connection=yes;'
)
        engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
        
        # 2. Data Extraction
        logging.info("Fetching data from SQL Server...")
        df = pd.read_sql("SELECT * FROM v_FinancialAnalysis", engine)
       
        
        # 3. Simulation: Realistic Payment Delays
        np.random.seed(42) 
        delays = np.random.gamma(shape=2, scale=5, size=len(df)).astype(int) - 3
        

        df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')
        df['DueDate'] = pd.to_datetime(df['DueDate'], errors='coerce')
        df['ActualPaymentDate'] = df['DueDate'] + pd.to_timedelta(delays, unit='D')

        # 4. Financial Calculations: Aging & DSO
        df['DaysToPay'] = (df['ActualPaymentDate'] - df['OrderDate']).dt.days

        def categorize_aging(days):
            if days <= 0: 
                return 'On Time'
            elif days <= 30: 
                return '1-30 Days Late'
            elif days <= 60: 
                return '31-60 Days Late'
            else: 
                return 'Over 60 Days'
        
        def sort_aging(days):
            if days <= 0: return 1
            elif days <= 30: return 2
            elif days <= 60: return 3
            else: return 4

        df['Delay_Category'] = (df['ActualPaymentDate'] - df['DueDate']).dt.days.apply(categorize_aging)
        df['Aging_Sort'] = (df['ActualPaymentDate'] - df['DueDate']).dt.days.apply(sort_aging)

        # 5. Export for Power BI
        try:
            df.to_csv('finance_data_ready.csv', index=False)
            logging.info("Success! ETL process complete. Data saved to CSV.")
        except PermissionError:
            logging.error("Permission Error: Please close the CSV file in Power BI before running.")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    run_etl_process()

2026-02-04 22:47:15,183 - INFO - Fetching data from SQL Server...
c:\Users\user\Desktop\projekty\venv\Lib\site-packages\pandas\io\sql.py:1649: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())
2026-02-04 22:47:16,216 - INFO - Success! ETL process complete. Data saved to CSV.
